## Data Visualization Course | VNU-HCM University of Science.
### Final Project
#### **STEP 02 - DATA Preprocessing**

In [48]:
import sys
import os
shared_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'Shared'))
sys.path.append(shared_path)

In [49]:
from Libraries import *
from Shared_Functions import *

In [50]:
dataset_source_path = getGranDir() / 'Dataset'
pre_process_dataset_path = dataset_source_path / 'scores.csv'
raw_df = pd.read_csv(pre_process_dataset_path)
raw_df.head()

,Student ID,Mathematics,Literature,Foreign language,Physics,Chemistry,Biology,History,Geography,Civic education,Foreign language code
0,1000001,8.4,8.50,9.2,NaN,NaN,NaN,6.75,6.00,9.00,N1
1,1000002,7.2,8.50,9.2,NaN,NaN,NaN,8.75,6.50,8.50,N1
2,1000003,NaN,6.50,NaN,NaN,NaN,NaN,9.25,7.50,NaN,NaN
3,1000004,7.8,8.25,7.8,NaN,NaN,NaN,4.50,6.25,8.25,N1
4,1000005,7.2,8.00,7.8,NaN,NaN,NaN,4.75,6.75,8.25,N1


**1. Kiểm tra kiểu dữ liệu và số lượng hàng, cột**

In [51]:
print(raw_df.dtypes)
raw_df.shape

Student ID                 int64
Mathematics              float64
Literature               float64
Foreign language         float64
Physics                  float64
Chemistry                float64
Biology                  float64
History                  float64
Geography                float64
Civic education          float64
Foreign language code     object
dtype: object


(1022060, 11)

**Nhận xét:** Các cột đã có kiểu dữ liệu phù hợp với ý nghĩa.

**2. Kiểm tra điều kiện tồn tại của các cột.**

**2.1 Kiểm tra cột ID: các ID này phải là DUY NHẤT, và phải lớn hơn 0.**

In [52]:
# ID
ID_col = raw_df['Student ID']
less_than_0 = ID_col[ID_col < 0]
duplicate_values = ID_col[ID_col.duplicated(keep=False)]
print('Less than zero: ')
print(less_than_0)
print('Values is not unique: ')
print(duplicate_values)

Less than zero: 
Series([], Name: Student ID, dtype: int64)
Values is not unique: 
Series([], Name: Student ID, dtype: int64)


**Nhận xét:** Như vậy không có giá trị nào bé hơn không và không có giá trị nào bị trùng lập.

**Kết luận:** Vì không có trùng lặp cho nên không có hàng nào trong dataset sẽ bị trùng lập.

**2.2 Kiểm tra các cột điểm - score: Score phải >= 0 và <= 10 hoặc bằng NaN.**

In [53]:
score_df = raw_df.select_dtypes(include='float64')
score_df.head()
for col in score_df.columns:
    less_than_0 = score_df[score_df[col] < 0]
    greater_than_10 = score_df[score_df[col] > 10]
    
    if len(less_than_0.index.tolist()):
        print(f'Column: {col}')
        print('Less than zero (indices): ')
        print(less_than_0.index.tolist())
    
    if len(greater_than_10.index.tolist()):
        print(f'Column: {col}')
        print('Greater than ten (indices): ')
        print(greater_than_10.index.tolist())

**Nhận xét:** Không có output được in ra, tức nghĩa là các cột đều đã thõa mãn điều kiện.

**2.4 Kiểm tra cột có kiểu dữ liệu không phải dữ liệu kiểu số**

Ở đây ta chỉ cần tìm ra các trường DUY NHẤT.

In [54]:
non_numerical = raw_df['Foreign language code']
unique_values = non_numerical.unique()
unique_values

array(['N1', nan, 'N6', 'N4', 'N7', 'N5', 'N3', 'N2'], dtype=object)

Kiểm tra số lượng các loại ngôn ngữ được thi dựa trên mã ngôn ngữ

In [55]:
foreign_language_counts = raw_df['Foreign language code'].value_counts()
print (foreign_language_counts)

Foreign language code
N1    876102
N4      2940
N3       716
N6       659
N7       407
N2        87
N5        86
Name: count, dtype: int64


**3. Kiểm tra các thiếu khuyết trong dữ liệu:**

In [56]:
def calcMissingRatio(df):
    missing_ratio = {}
    for key, values in df.items():
        missing_count = values.isna().sum()
        total_count = len(values)
        missing_ratio[key] = round(( missing_count / total_count )* 100, 1) if total_count != 0 else None
    return missing_ratio

In [57]:
missing_ratio = calcMissingRatio(raw_df)
for key, values in missing_ratio.items():
    print(f'Column: {key} - Missing Ratio: {values}')

Column: Student ID - Missing Ratio: 0.0
Column: Mathematics - Missing Ratio: 1.8
Column: Literature - Missing Ratio: 1.4
Column: Foreign language - Missing Ratio: 13.8
Column: Physics - Missing Ratio: 68.0
Column: Chemistry - Missing Ratio: 67.9
Column: Biology - Missing Ratio: 68.2
Column: History - Missing Ratio: 33.1
Column: Geography - Missing Ratio: 33.3
Column: Civic education - Missing Ratio: 44.7
Column: Foreign language code - Missing Ratio: 13.8


**Nhận xét:**
1. Ta thấy các cột như: `Mathematics`, `Literature`, `Foreign language` có tỉ lệ thiếu khuyết thấp, biểu thị rằng các môn học này rất quan trọng.
2. Các môn học còn lại  `Physics`, `Chemistry`, `Biology`, `History`, `Geography`, `Civic education` đều có thiếu khuyết. Điều này xảy ra là bởi vì các thí sinh chọn những khối thi khác nhau.
3. Cột như `Student ID` có tỉ lệ thiếu khuyết là 0.0%.
4. Loại ngôn ngữ được thi nhiều nhất có mã là N1, đây là môn Tiếng Anh.


**Kết luận:** 
1. Ta sẽ xét các môn thi chính thức như Toán - Lý - Hóa - Sinh - Văn - Sử - Địa - GDCD. 
2. Đối với môn thi ngoại ngữ, ta sẽ chọn môn Tiếng Anh để xét vì độ phổ biến của nó
3. Vì do chúng ta đã xóa các cột ngoại ngữ khác nên cột định nghĩa còn 1 loại là N1, ta sẽ xóa luôn cột định nghĩa ký hiệu cho các loại ngoại ngữ : `Foreign language code`

In [58]:
filtered_df = raw_df[raw_df['Foreign language code'] == 'N1']
filtered_df = filtered_df.drop(columns=['Foreign language code'])

row_count = filtered_df.shape[0]
print(f"Number of rows with 'Foreign language code' as 'N1': {row_count}")

filtered_df.head()

Number of rows with 'Foreign language code' as 'N1': 876102


,Student ID,Mathematics,Literature,Foreign language,Physics,Chemistry,Biology,History,Geography,Civic education
0,1000001,8.4,8.50,9.2,NaN,NaN,NaN,6.75,6.00,9.00
1,1000002,7.2,8.50,9.2,NaN,NaN,NaN,8.75,6.50,8.50
3,1000004,7.8,8.25,7.8,NaN,NaN,NaN,4.50,6.25,8.25
4,1000005,7.2,8.00,7.8,NaN,NaN,NaN,4.75,6.75,8.25
5,1000006,7.6,9.25,9.0,NaN,NaN,NaN,8.50,7.25,9.75


**4. Có hai khối thi là Khoa học tự nhiên và Khoa học xã hội, khi ta thi KHTN ( lý - hóa - sinh ) thì sẽ không thi các môn KHXH ( sử - địa - GDCD ).**

Chính vì vậy mà ta sẽ không tiến hành điền khuyết NaN vào trong các trường dữ liệu này. Mà ta sẽ phân tích dựa theo hai tổ hợp môn này.

**5. Sắp xếp các kết quả hàng theo id.**

In [59]:
filtered_df.sort_values(by='Student ID', ascending=True, inplace=True)
raw_df = filtered_df.copy()
filtered_df.head()

,Student ID,Mathematics,Literature,Foreign language,Physics,Chemistry,Biology,History,Geography,Civic education
0,1000001,8.4,8.50,9.2,NaN,NaN,NaN,6.75,6.00,9.00
1,1000002,7.2,8.50,9.2,NaN,NaN,NaN,8.75,6.50,8.50
3,1000004,7.8,8.25,7.8,NaN,NaN,NaN,4.50,6.25,8.25
4,1000005,7.2,8.00,7.8,NaN,NaN,NaN,4.75,6.75,8.25
5,1000006,7.6,9.25,9.0,NaN,NaN,NaN,8.50,7.25,9.75


**6. Tiến hành ghi tập dữ liệu sau khi xử lý vào file csv.**

In [60]:
processed_dataset_path = dataset_source_path / 'THPTQG_2023_processed.csv'
filtered_df.to_csv(processed_dataset_path, index=False)